In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

C:\Users\tsmar\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [274]:
ComicDataDC = pd.read_csv("C:/Users/tsmar/dc-wikia-data.csv")
ComicDataMarvel = pd.read_csv("C:/Users/tsmar/marvel-wikia-data.csv")

In [275]:
ComicDataDC.fillna(value=0, axis=0, inplace=True)
ComicDataMarvel.fillna(value=0, axis=0, inplace=True)

In [276]:
ComicData_DC = ComicDataDC.drop(labels=['urlslug', 'page_id', 'GSM', 'FIRST APPEARANCE', 'ALIGN'], axis=1) #drop unwanted cols
ComicData_Marvel = ComicDataMarvel.drop(labels=['urlslug', 'page_id', 'GSM', 'FIRST APPEARANCE', 'ALIGN'], axis=1)

#ComicData3 = ComicData2.fillna(0, axis=0) #drop rows with NaN values.
Target_DC = ComicDataDC.iloc[:, 4]
Target_Marvel = ComicDataMarvel.iloc[:, 4]

In [277]:
len(Target_DC)
len(Target_Marvel)

16376

In [278]:
#we will be prediction the alignment of a superhero based on other data about them. 

In [279]:
#now, this dataframe needs to be converted into numerical data so that machine learning can be run on it. 
#lets build some dictionaries!!!!
eye_map = {'Blue Eyes' : 1, 'Green Eyes' : 2, 'Brown Eyes' : 3, 'Black Eyes' : 4, 'Red Eyes' : 5, 'White Eyes' : 6,
           'Hazel Eyes' : 7,  'Yellow Eyes' : 8, 'Purple Eyes' : 9, 'Orange Eyes' : 10, 'Pink Eyes' : 11, 'Silver Eyes' : 12,
           'Gold Eyes' : 13, 'Grey Eyes' : 14, 'Photocellular Eyes' : 15, 'Amber Eyes' : 16, 'Violet Eyes':17, 'Auburn Hair':20, 'Black Eyeballs': 4, 
           0:0, 'Variable Eyes':21, 'One Eye':22, 'Multiple Eyes' : 23, 'Magenta Eyes':17, 'Yellow Eyeballs' : 8, 'No Eyes':24,
          'Compound Eyes' : 25}

hair_map = {'Blue Hair' : 1, 'Green Hair' : 2, 'Brown Hair' : 3, 'Black Hair' : 4, 'Red Hair' : 5, 'White Hair' : 6,
           'Hazel Hair' : 7,  'Yellow Hair' : 8, 'Purple Hair' : 9, 'Orange Hair' : 10, 'Pink Hair' : 11, 'Silver Hair' : 12, 
            'Blond Hair' : 13, 'Strawberry Blond Hair' : 14, 'Gold Hair' : 15, 'Grey Hair' : 16, 'Reddish Brown Hair':17, 'Violet Hair':18,
            'Platinum Blond Hair':19, 'Auburn Hair':20, 0:0, 'No Hair':21, 'Bald' : 21, 'Reddish Blond Hair' : 14,
           'Variable Hair': 23, 'Light Brown Hair' : 3, 'Magenta Hair' : 9, 'Bronze Hair' : 24, 'Dyed Hair' : 23,
           'Orange-brown Hair': 25}

In [280]:
id_map = {'Secret Identity' : 0, 'Public Identity' : 1, 'Identity Unknown': 2, 'No Dual Identity':3, 'Known to Authorities Identity':4, 
          0:0}
gender_map = {'Male Characters' : 0, 'Female Characters' : 1, 'Genderless Characters': 2, 'Transgender Characters': 3,'Genderfluid Characters': 4, 'Agender Characters':5, 0:0}
alive_map = {'Living Characters' : 0, 'Deceased Characters' : 1, 0:0}
align_map = {'Good Characters' : 0, 'Neutral Characters' : 1, 'Bad Characters' : 2, 'Reformed Criminals' : 3, 0:4}

In [281]:
#I will predict if a character is aligned to Good or Evil or Reformed based on these following data items:
    #1. Gender,
    #2. Eye Color
    #3. is Alive
    #4. Number of Appearences
    #5. Year
    #6. simple hash of their name.
    #7. hair color. 

In [282]:
ComicData = pd.concat([ComicData_DC, ComicData_Marvel], ignore_index=True, axis=0)
ComicAllignment = pd.concat([Target_DC, Target_Marvel], ignore_index=True, axis=0)

ComicAllignment
ComicData.head

C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


<bound method NDFrame.head of                      ALIVE  APPEARANCES         EYE        HAIR  \
0        Living Characters       3093.0   Blue Eyes  Black Hair   
1        Living Characters       2496.0   Blue Eyes  Black Hair   
2        Living Characters       1565.0  Brown Eyes  Brown Hair   
3        Living Characters       1316.0  Brown Eyes  White Hair   
4        Living Characters       1237.0   Blue Eyes  Black Hair   
5        Living Characters       1231.0   Blue Eyes  Black Hair   
6        Living Characters       1121.0   Blue Eyes  Blond Hair   
7        Living Characters       1095.0   Blue Eyes  Black Hair   
8        Living Characters       1075.0   Blue Eyes  Blond Hair   
9        Living Characters       1028.0   Blue Eyes  Blond Hair   
10       Living Characters       1028.0   Blue Eyes  Blond Hair   
11     Deceased Characters        969.0   Blue Eyes  Blond Hair   
12       Living Characters        951.0   Blue Eyes    Red Hair   
13       Living Characters      

In [283]:
#first we will turn list of alignments into list of numbers: 
import math
num_align = [align_map[d] for d in ComicAllignment] #these are our labels. 
num_gender = [gender_map[g[5]] for g in ComicData.values]
num_eye_color = [eye_map[c[2]] for c in ComicData.values]
num_isAlive = [alive_map[a[0]] for a in ComicData.values]
num_hair = [hair_map[h[3]] for h in ComicData.values]
num_id = [id_map[p[4]] for p in ComicData.values]
num_name = [len(word[8]) for word in ComicData.values]

In [284]:
ComicData.values[0]

array(['Living Characters', 3093.0, 'Blue Eyes', 'Black Hair',
       'Secret Identity', 'Male Characters', 1939.0, nan,
       'Batman (Bruce Wayne)'], dtype=object)

In [285]:
ComicData['ID'] = num_id
ComicData['EYE'] = num_eye_color
ComicData['ALIVE'] = num_isAlive
ComicData['HAIR'] = num_hair
ComicData['SEX'] = num_gender
ComicData['name'] = num_name
ComicData.drop(labels=['Year', 'YEAR'], axis=1, inplace=True)
ComicData.head

<bound method NDFrame.head of        ALIVE  APPEARANCES  EYE  HAIR  ID  SEX  name
0          0       3093.0    1     4   0    0    20
1          0       2496.0    1     4   0    0    21
2          0       1565.0    3     3   0    0    26
3          0       1316.0    3     6   1    0    24
4          0       1237.0    1     4   0    0    27
5          0       1231.0    1     4   1    1    27
6          0       1121.0    1    13   1    0    22
7          0       1095.0    1     4   0    0    25
8          0       1075.0    1    13   1    1    30
9          0       1028.0    1    13   0    0    19
10         0       1028.0    1    13   0    1    10
11         1        969.0    1    13   0    0    22
12         0        951.0    1     5   0    1    26
13         0        951.0    1     3   1    0    25
14         0        934.0    1     4   1    1    21
15         0        930.0    1     4   1    0    29
16         0        803.0    1     3   0    0    23
17         0        716.0    2    

In [286]:

ComicData.head

<bound method NDFrame.head of        ALIVE  APPEARANCES  EYE  HAIR  ID  SEX  name
0          0       3093.0    1     4   0    0    20
1          0       2496.0    1     4   0    0    21
2          0       1565.0    3     3   0    0    26
3          0       1316.0    3     6   1    0    24
4          0       1237.0    1     4   0    0    27
5          0       1231.0    1     4   1    1    27
6          0       1121.0    1    13   1    0    22
7          0       1095.0    1     4   0    0    25
8          0       1075.0    1    13   1    1    30
9          0       1028.0    1    13   0    0    19
10         0       1028.0    1    13   0    1    10
11         1        969.0    1    13   0    0    22
12         0        951.0    1     5   0    1    26
13         0        951.0    1     3   1    0    25
14         0        934.0    1     4   1    1    21
15         0        930.0    1     4   1    0    29
16         0        803.0    1     3   0    0    23
17         0        716.0    2    

In [287]:
#so we have our labels: 
len(num_align)

23272

In [288]:
#Now the data is ready! we can go to work!
#lets split up the data into training, validation, testing
#now i will also add normalization!
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

TrainSplit = 5/8
ValidSplit = 1/8
TestSplit = 2/8

data_size = 23272
train_len = int(data_size * TrainSplit)
valid_len = int(data_size * ValidSplit)
test_len = int(data_size * TestSplit)

TrainSet = mms.fit_transform(ComicData.values[:train_len])
TrainLabels = num_align[:train_len]

ValidationSet = mms.transform(ComicData.values[train_len+1:train_len+valid_len+1])
ValidationLabels = num_align[train_len+1:train_len+valid_len+1]

TestSet = mms.transform(ComicData.values[train_len + valid_len:])
TestLabels = num_align[train_len + valid_len:]

In [289]:
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.layers import Input, Dense, Activation, Dropout
from keras.models import Sequential
from keras.models import Model

In [290]:
model = Sequential()
model.add(Dense(64, input_dim=7, activation='relu', init='uniform'))
model.add(Dropout(rate=0.125))
model.add(Dense(32, activation='relu', init='uniform'))
model.add(Dropout(rate=0.125))
model.add(Dense(16, activation='relu', init='uniform'))
model.add(Dropout(rate=0.125))
model.add(Dense(8, activation='relu', init='uniform'))
model.add(Dense(5, activation='softmax', init='uniform'))
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, input_dim=7, activation="relu", kernel_initializer="uniform")`
  
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(16, activation="relu", kernel_initializer="uniform")`
  
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
  
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(5, activation="softmax", kernel_initializer="uniform")`
  

In [291]:
len(TrainLabels)

14545

In [292]:
TrainSet = np.reshape(TrainSet, (train_len, 7))
TrainLabelSet = []
for t in TrainLabels:
    if t == 0:
        TrainLabelSet.append([1, 0, 0, 0, 0])
    elif t == 1:
        TrainLabelSet.append([0, 1, 0, 0, 0])
    elif t == 2:
        TrainLabelSet.append([0, 0, 1, 0, 0])
    elif t == 3:
        TrainLabelSet.append([0, 0, 0, 1, 0])
    elif t == 4:
        TrainLabelSet.append([0, 0, 0, 0, 1])
ValidationSet = np.reshape(ValidationSet, (valid_len, 7))
ValidLabelSet = []
for t in ValidationLabels:
    if t == 0:
        ValidLabelSet.append([1, 0, 0, 0, 0])
    elif t == 1:
        ValidLabelSet.append([0, 1, 0, 0, 0])
    elif t == 2:
        ValidLabelSet.append([0, 0, 1, 0, 0])
    elif t == 3:
        ValidLabelSet.append([0, 0, 0, 1, 0])
    elif t == 4:
        ValidLabelSet.append([0, 0, 0, 0, 1])
        
TestSet = np.reshape(TestSet, (test_len, 7))
TestLabelSet = []
for t in TestLabels:
    if t == 0:
        TestLabelSet.append([1, 0, 0, 0, 0])
    elif t == 1:
        TestLabelSet.append([0, 1, 0, 0, 0])
    elif t == 2:
        TestLabelSet.append([0, 0, 1, 0, 0])
    elif t == 3:
        TestLabelSet.append([0, 0, 0, 1, 0])
    elif t == 4:
        TestLabelSet.append([0, 0, 0, 0, 1])

In [293]:
model.fit(TrainSet, np.array(TrainLabelSet), epochs=10, verbose=1)

Epoch 1/10
14545/14545 [==============================] - 4s 256us/step - loss: 1.2920 - acc: 0.3912
Epoch 2/10
14545/14545 [==============================] - ETA: 0s - loss: 1.2295 - acc: 0.394 - 1s 84us/step - loss: 1.2289 - acc: 0.3946
Epoch 3/10
14545/14545 [==============================] - 1s 81us/step - loss: 1.2242 - acc: 0.4091
Epoch 4/10
14545/14545 [==============================] - 1s 84us/step - loss: 1.2187 - acc: 0.4421
Epoch 5/10
14545/14545 [==============================] - 1s 89us/step - loss: 1.2123 - acc: 0.4668
Epoch 6/10
14545/14545 [==============================] - 1s 87us/step - loss: 1.2031 - acc: 0.4842
Epoch 7/10
14545/14545 [==============================] - 1s 86us/step - loss: 1.1972 - acc: 0.4876
Epoch 8/10
14545/14545 [==============================] - 1s 82us/step - loss: 1.1924 - acc: 0.4883
Epoch 9/10
14545/14545 [==============================] - 1s 87us/step - loss: 1.1861 - acc: 0.4989
Epoch 10/10
14545/14545 [==============================] - 1s

In [294]:
scores = model.evaluate(ValidationSet, np.array(ValidLabelSet))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

2909/2909 [==============================] - 1s 235us/step

acc: 43.73%


In [295]:
scores = model.evaluate(TestSet, np.array(TestLabelSet))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

5818/5818 [==============================] - 0s 41us/step

acc: 44.24%


In [297]:
#now lets try a randomforest
import sklearn
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy', n_estimators=100, random_state=1, n_jobs=2)
forest.fit(TrainSet, TrainLabelSet)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [298]:
yhat = forest.predict(TestSet)

In [299]:
def label_match(yh, ya):
    for i in range(len(yh)):
        if yh[i] - ya[i] != 0.0:
            return False
    return True

In [300]:
res = [label_match(yhat[i], np.array(TestLabelSet[i])) for i in range(len(yhat))]

In [301]:
sum(res) / len(res)

0.3169474046063939